In [21]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
import time
import sys


In [28]:
csv_path = "../data/player_add_info_errored_recs.csv"
bb_players_df = pd.read_csv(csv_path)
bb_players_df.head(10)

,Unnamed: 0,player_name,year,team,player_position,player_height,player_weight,process_status
0,271,Brian McDonald,2009,Northeastern,-1,-1,-1,Failure: Did not find player
1,707,Marques Johnson,2009,Boston University,-100,-100,-100,Failure: Other Error
2,800,Al Fisher,2009,Kent St.,-1,-1,-1,Failure: Did not find player
3,904,Ifeanyi Ehirim,2009,Robert Morris,-1,-1,-1,Failure: Did not find player
4,921,Phillip Adams,2009,South Carolina St.,-1,-1,-1,Failure: Did not find player
5,1114,Olawale Alesh,2009,Rhode Island,-1,-1,-1,Failure: Did not find player
6,1254,Anthony Jones,2009,Savannah St.,-100,-100,-100,Failure: Other Error
7,1591,Jason Aycock,2009,UNC Wilmington,-1,-1,-1,Failure: Did not find player
8,1844,Alex English,2009,Colgate,-100,-100,-100,Failure: Other Error
9,2061,Jeff Kari,2009,Lafayette,-100,-100,-100,Failure: Other Error


In [51]:
year=[]
team=[]
player_name = []
player_height = []
player_weight = []
player_position = []
process_status = []
start_row=0
end_row=10000
for index, row in bb_players_df.iterrows():
    height=0
    weight=0
    position="NA"
#    print(index,"-" ,row['player_name'])
    if index < start_row:
        continue
    if index > end_row:
        break
    player = row['player_name'].lower().replace(".","").replace("'","").replace(",","").replace("`","")
    first_last = player.split()
    sports_ref_player =""
    for windex,word in enumerate(first_last):
        if windex==0 and len(first_last) > 1:
            sports_ref_player = word
        elif windex==0 and len(first_last) == 1:
            sports_ref_player = word + "-1"
        elif windex > 0 and len(first_last) > windex +1:
            sports_ref_player = sports_ref_player + "-" +   word
        elif windex > 0 and len(first_last) == windex+1:
            sports_ref_player = sports_ref_player + "-" +   word + "-1"
            
    print(sports_ref_player)
#    sports_ref_player = first_last[0] + "-" + first_last[1] + "-1"
    milliseconds = 100
    seconds = 0.001 * milliseconds
    start_time = time.time()
    time.sleep(seconds)
    stop_time = time.time()
#    time.sleep(1)
    player_name.append(row['player_name'])
    year.append(row['year'])
    team.append(row['team'])
    url = 'https://www.sports-reference.com/cbb/players/' + sports_ref_player+'.html'
    try:
        response = requests.get(url)
#        print("received----------------------")
#        print(response.text)
#        print("received----------------------")
#        print(response.text)
        if response.text.find("Position:") > -1 and response.text.find('itemprop="height"') > -1 :
            print("Found url")
        else :
            response = requests.get(url.replace("-1","-2"))
            
        soup = BeautifulSoup(response.text, 'html.parser')
#        print(soup)
        if response.text.find("Position:") > -1 and response.text.find('itemprop="height"') > -1 : 
#            print("Looking for position now")
            results = soup.find_all('div', id='info')
            #print(results[0])
            ptags = results[0].find_all('p')
            for ptag in ptags:
#                print("-",ptag.text,"-")
                if ptag.text.find("Position") > -1:
                    position = ptag.contents[2].strip()
                    print("Position :", position)
                    poop = ptag.contents[3].contents[-1].strip().replace("(","").replace(")","").replace("cm","").replace("kg","")
                    print(poop)
                
            #print(ptags)
#            print(ptags[0].contents[2].strip())
#            position=ptags[0].contents[2].strip()
#            print('Height tags',ptags[0].contents[3].contents[-1].strip())
#            print(ptags[0].contents[3].contents[-1].strip())
#            poop = ptags[0].contents[3].contents[-1].strip().replace("(","").replace(")","").replace("cm","").replace("kg","")
            print(poop)
            height_weight= poop.split(",\xa0")
            if len(height_weight) > 1 :
                height=height_weight[0]
                weight=height_weight[1]
            elif len(height_weight) > 0 :
                height=height_weight[0]
            else:
                height=-200
                weight=-200
            
#            height,weight = poop.split(",\xa0")
            print(height,"-" ,weight)
            player_position.append(position)
#            print('appended position:',len(player_position))
            player_height.append(height)
            player_weight.append(weight)
#            print('appended weight:',len(player_weight))
            process_status.append("Success")
        else:
            print("Not Found in ")
#            print("Not Found in :",response.text)
            process_status.append("Failure: Did not find player")
            player_position.append(-1)
            player_height.append(-1)
            player_weight.append(-1)
    except:
            (type, value, tb) = sys.exc_info()
            err = "%s" % value
            process_status.append("Failure: Other Error")
            print("Other Error")
            player_position.append(-100)
            player_height.append(-100)
            player_weight.append(-100)
    

brian-mcdonald-1
Not Found in 
marques-johnson-1
Found url
Position : Forward
201, 98
201, 98
201 - 98
al-fisher-1
Not Found in 
ifeanyi-ehirim-1
Not Found in 
phillip-adams-1
Not Found in 
olawale-alesh-1
Not Found in 
anthony-jones-1
Found url
Position : Guard
198, 88
198, 88
198 - 88
jason-aycock-1
Not Found in 
alex-english-1
Found url
Position : Forward
201, 86
201, 86
201 - 86
jeff-kari-1
Found url
Position : Guard
188, 83
188, 83
188 - 83
john-williams-1
Found url
Position : Forward
203, 106
203, 106
203 - 106
ryan-butler-1
Not Found in 
conroy-witter-1
Not Found in 
alfred-jones-1
Not Found in 
wes-libuit-1
Not Found in 
blaine-booher-1
Not Found in 
jarrod-randall-1
Not Found in 
sheyi-olaoshebikan-1
Other Error
tramaine-pride-1
Not Found in 
yannick-braimbridge-1
Not Found in 
dion-williams-1
Not Found in 
nick-scott-1
Not Found in 
marc-ellis-1
Not Found in 
jerime-anderson-1
Not Found in 
yonas-berhe-1
Not Found in 
tyler-maddry-1
Not Found in 
sam-shew-1
Not Found in 
jame

Found url
Position : Guard
196, 88
196, 88
196 - 88
marius-chawa-1
Not Found in 
matt-cajuste-1
Not Found in 
darrell-tripp-1
Not Found in 
dion-sims-1
Not Found in 
c-j-collins-1
Not Found in 
kenny-lawson-jr-1
Not Found in 
jaime-sema-1
Not Found in 
t-vazquez-simmons-1
Not Found in 
justin-roper-1
Not Found in 
joe-bucher-1
Not Found in 
michael-loyd-jr-1
Not Found in 
marques-johnson-1
Found url
Position : Forward
201, 98
201, 98
201 - 98
drew-godwin-1
Not Found in 
quintin-dailey-1
Found url
Position : Guard
190, 81
190, 81
190 - 81
donte-lopez-1
Not Found in 
david-buchannon-1
Not Found in 
anthony-mason-jr-1
Not Found in 
tony-white-jr-1
Not Found in 
zach-beach-1
Not Found in 
jameel-forbes-1
Not Found in 
morris-smith-iv-1
Not Found in 
chioke-walker-1
Not Found in 
peter-zesterman-1
Not Found in 
kenny-franklin-jr-1
Not Found in 
peter-abraham-1
Not Found in 
andres-irarrazabal-1
Not Found in 
david-(dj)-jackson-1
Not Found in 
luuk-kortekass-1
Not Found in 
dwight-thorne-ii-

Not Found in 
jerime-anderson-1
Not Found in 
humpty-hitchens-1
Not Found in 
mike-myers-keitt-1
Not Found in 
dee-brown-1
Found url
Position : Guard
185, 72
185, 72
185 - 72
anthony-jones-1
Found url
Position : Guard
198, 88
198, 88
198 - 88
jaime-sema-1
Not Found in 
savalace-townsend-1
Not Found in 
jordan-henriquez-1
Not Found in 
quintin-dailey-1
Found url
Position : Guard
190, 81
190, 81
190 - 81
shaquil-barber-1
Not Found in 
griffin-hoffmann-1
Not Found in 
roderick-pearson-jr-1
Not Found in 
ralph-sampson-iii-1
Not Found in 
jonathon-thompson-1
Not Found in 
rob-wilson-1
Not Found in 
douglas-davis-1
Not Found in 
tashan-fredrick-1
Not Found in 
rusian-pateev-1
Not Found in 
derrick-thomas-1
Not Found in 
reggie-johnson-1
Found url
Position : Forward
206, 92
206, 92
206 - 92
jorge-brian-diaz-1
Not Found in 
verdell-jones-iii-1
Not Found in 
khaliff-wyatt-1
Not Found in 
connor-moore-1
Not Found in 
kenny-mcgowen-1
Not Found in 
stormberg-taylor-1
Not Found in 
keith-mcdougald-

Not Found in 
frantz-massenat-1
Not Found in 
pieter-prinsloo-1
Not Found in 
babatunde-olumuyiwa-1
Not Found in 
danny-felmann-1
Not Found in 
xavier-blount-1
Not Found in 
adrian-rodgers-1
Not Found in 
lowell-ulmer-1
Not Found in 
marshall-harris-iii-1
Not Found in 
larry-nance-jr-1
Not Found in 
raph-thomas-edwards-1
Not Found in 
dave-whitmore-1
Not Found in 
michael-hale-iii-1
Not Found in 
josh-mason-1
Not Found in 
rahlir-hollis-jefferson-1
Not Found in 
jesus-delgado-1
Not Found in 
tre-drye-1
Not Found in 
collin-chiverton-1
Not Found in 
jourdan-demuynck-1
Not Found in 
warren-flood-jr-1
Not Found in 
jared-accetturra-1
Not Found in 
dominque-major-1
Not Found in 
shawn-kemp-jr-1
Not Found in 
mike-caffey-1
Not Found in 
alexander-barlow-1
Not Found in 
zach-lamb-1
Not Found in 
johnny-obryant-iii-1
Not Found in 
raymond-cowels-iii-1
Not Found in 
joey-ptasinski-1
Not Found in 
john-walton-1
Not Found in 
deji-ibitayo-1
Not Found in 
bisi-addey-1
Not Found in 
jonathan-mills

Not Found in 
jomari-bradshaw-1
Not Found in 
walter-davis-1
Position : Forward
198, 87
198, 87
198 - 87
anthony-myers-pate-1
Not Found in 
clyde-smith-iii-1
Not Found in 
joshua-freelove-1
Not Found in 
duke-antoneli-1
Not Found in 
derrick-richardson-jr-1
Not Found in 
marco-porcher-jimenez-1
Not Found in 
sidney-sanders-jr-1
Not Found in 
lloyd-neely-ii-1
Not Found in 
desi-washington-1
Not Found in 
matt-ratto-1
Not Found in 
reggie-johnson-1
Found url
Position : Forward
206, 92
206, 92
206 - 92
anthony-white-jr-1
Not Found in 
zachary-burnham-1
Not Found in 
ali-fall-1
Not Found in 
adama-adams-1
Not Found in 
dion-wright-1
Not Found in 
carlos-lopez-sosa-1
Not Found in 
mike-mccall-jr-1
Not Found in 
geoffrey-groselle-1
Not Found in 
zieyik-estimă¨-1
Not Found in 
dez-wells-1
Not Found in 
moses-ayegba-1
Not Found in 
emmanual-okoro-1
Not Found in 
alhaji-fullah-1
Not Found in 
traddarius-mcphearson-1
Not Found in 
raymond-penn-1
Not Found in 
barrington-stevens-iii-1
Not Found i

...

Not Found in 
jairus-stevens-1
Not Found in 
ibrahim-famouke-doumbia-1
Not Found in 
dwayne-rose-jr-1
Not Found in 
mitch-ballock-1
Not Found in 
marcus-shaver-jr-1
Not Found in 
anthony-oliver-ii-1
Not Found in 
jermaine-blackmon-1
Not Found in 
david-jenkins-jr-1
Not Found in 
dallas-polk-hillia-1
Not Found in 
derek-webster-jr-1
Not Found in 
cedric-council-jr-1
Not Found in 
jaylen-mccoy-1
Not Found in 
tj-shorts-ii-1
Not Found in 
jonathan-bryant-ii-1
Not Found in 
keaton-van-soelen-1
Not Found in 
jonathan-nwankwo-1
Not Found in 
derrick-walker-jr-1
Not Found in 
michael-porter-jr-1
Not Found in 
jay-jay-chandler-1
Not Found in 
mack-smith-1
Not Found in 
terry-ellis-jr-1
Not Found in 
johnny-crnogorac-1
Not Found in 
chris-joyce-1
Not Found in 
anthony-longpre-1
Not Found in 
chris-harris-jr-1
Not Found in 
james-west-iv-1
Not Found in 
leroy-butts-iv-1
Not Found in 
nick-weatherspoon-1
Found url
Position : Forward
201, 88
201, 88
201 - 88
gary-trent-jr-1
Not Found in 
royce-ham

Not Found in 
mack-smith-1
Not Found in 
johnny-crnogorac-1
Not Found in 
chris-joyce-1
Not Found in 
charlie-obriant-1
Not Found in 
anthony-longpre-1
Not Found in 
chris-harris-jr-1
Not Found in 
james-west-iv-1
Not Found in 
leroy-butts-iv-1
Not Found in 
nick-weatherspoon-1
Found url
Position : Forward
201, 88
201, 88
201 - 88
royce-hamm-jr-1
Not Found in 
matt-coleman-iii-1
Not Found in 
jordan-roberts-1
Not Found in 
terrell-gomez-1
Not Found in 
greg-bowie-ii-1
Not Found in 
charles-obannon-jr-1
Not Found in 
brian-coffey-ii-1
Not Found in 
john-grant-jr-1
Not Found in 
derrik-jamerson-jr-1
Not Found in 
damonte-williams-1
Not Found in 
hunt-conroy-1
Not Found in 
kevin-johnson-1
Found url
Position : Center
185, 81
185, 81
185 - 81
cheikh-mbacke-diong-1
Not Found in 
marvin-nesbitt-jr-1
Not Found in 
mckinley-wright-iv-1
Not Found in 
curtis-haywood-ii-1
Not Found in 
victor-bailey-jr-1
Not Found in 
jahshire-hardnett-1
Not Found in 
kz-okpala-1
Not Found in 
shaq-buchanan-1
Not

Not Found in 
jerry-johnson-jr-1
Not Found in 
loren-cristian-jackson-1
Not Found in 
james-love-iii-1
Not Found in 
john-edgar-jr-1
Not Found in 
tj-gibbs-1
Not Found in 
ed-morrow-1
Not Found in 
danny-harris-jr-1
Not Found in 
travis-fields-jr-1
Not Found in 
clayton-le-sann-1
Not Found in 
devin-morgan-1
Not Found in 
samuel-japhet-mathias-1
Not Found in 
sabastian-townes-1
Not Found in 
justin-cole-1
Not Found in 
lavar-batts-jr-1
Not Found in 
levi-stockard-iii-1
Not Found in 
luka-kraljevic-1
Not Found in 
ty-jones-1
Not Found in 
phlandrous-fleming-jr-1
Not Found in 
jermaine-jackson-jr-1
Not Found in 
jamil-jackson-jr-1
Not Found in 
willy-nunez-jr-1
Not Found in 
trent-frazier-1
Not Found in 
cedrick-alley-jr-1
Not Found in 
fabian-white-jr-1
Not Found in 
denny-slay-ii-1
Not Found in 
stacy-beckton-jr-1
Not Found in 
jp-moorman-ii-1
Not Found in 
dwayne-rose-jr-1
Not Found in 
mitch-ballock-1
Not Found in 
aj-oliver-ii-1
Not Found in 
al-durham-1
Not Found in 
cameron-christ

Not Found in 
gary-harris-jr-1
Not Found in 
greg-milton-iii-1
Not Found in 
darus-maddox-1
Not Found in 
bernardo-da-silva-1
Not Found in 
matt-van-komen-1
Not Found in 
lance-singh-iii-1
Not Found in 
lance-coleman-ii-1
Not Found in 
marlon-taylor-1
Not Found in 
wendell-moore-jr-1
Not Found in 
vernon-carey-jr-1
Not Found in 
b-artis-white-1
Not Found in 
jonathan-dos-anjos-1
Not Found in 
arnold-henderson-vi-1
Not Found in 
chris-arcidiacono-1
Not Found in 
jelly-walker-1
Not Found in 
xaiver-armstead-1
Not Found in 
mario-mckinney-jr-1
Not Found in 
joseph-girard-iii-1
Not Found in 
danario-edgar-ii-1
Not Found in 
darin-green-jr-1
Not Found in 
ibrahim-famouke-doumbia-1
Not Found in 
oscar-lopez-jr-1
Not Found in 
eric-coleman-jr-1
Not Found in 
kwane-marble-ii-1
Not Found in 
george-dixon-1
Not Found in 
john-mccoy-1
Not Found in 
eddie-turner-iii-1
Not Found in 
dae-dae-grant-1
Not Found in 
wayne-bristol-jr-1
Not Found in 
james-edwards-1
Found url
Position : Center
213, 102
2

Not Found in 
rodney-carson-jr-1
Not Found in 
pj-roach-jr-1
Not Found in 
james-anderson-ii-1
Not Found in 
alonzo-verge-jr-1
Not Found in 
ford-cooper-jr-1
Not Found in 
scotty-pippen-jr-1
Not Found in 
blondeau-tchoukuiengo-1
Not Found in 
terrence-shannon-jr-1
Not Found in 
franklyn-penn-jr-1
Not Found in 
rafael-cruz-jr-1
Not Found in 
cedric-henderson-jr-1
Not Found in 
sean-east-ii-1
Not Found in 
tomas-woldetensae-1
Not Found in 
tavin-pierre-philippe-1
Not Found in 
chris-harris-jr-1
Not Found in 
avery-anderson-iii-1
Not Found in 
glenn-robinson-ii-1
Not Found in 
cj-gettelfinger-1
Not Found in 
james-glisson-iii-1
Not Found in 
jonathan-nunez-1
Not Found in 
chico-carter-jr-1
Not Found in 
charles-manning-jr-1
Not Found in 
jake-van-der-heijden-1
Not Found in 
jovan-blacksher-jr-1
Not Found in 
rudy-fitzgibbons-iii-1
Not Found in 
bernardo-da-silva-1
Not Found in 
japannah-kellogg-iii-1
Not Found in 
matt-van-komen-1
Not Found in 
lance-coleman-ii-1
Not Found in 
wendell-moo

In [52]:
print(player_position)
print(len(player_name),len(player_position),len(player_height),len(process_status))

[-1, 'Forward', -1, -1, -1, -1, 'Guard', -1, 'Forward', 'Guard', 'Forward', -1, -1, -1, -1, -1, -1, -100, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 'Forward', -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 'Guard', 'Guard', -1, -1, -1, -1, -1, 'Guard', -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 'Guard', -1, -1, -1, -1, -1, -1, -1, 'Guard', -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 'Guard

In [53]:
import numpy as np
player_add_info_dict = {"player_name":player_name,"year":year,"team":team,"player_position":player_position,"player_height":player_height,"player_weight":player_weight,"process_status":process_status}
player_add_info_df=pd.DataFrame(data=player_add_info_dict)
#process_status

#player_add_info_df["process_status"] = np.where(player_add_info_df["process_status"] == "Success", "Success", "Failure")
player_add_info_df

,player_name,year,team,player_position,player_height,player_weight,process_status
0,Brian McDonald,2009,Northeastern,-1,-1,-1,Failure: Did not find player
1,Marques Johnson,2009,Boston University,Forward,201,98,Success
2,Al Fisher,2009,Kent St.,-1,-1,-1,Failure: Did not find player
3,Ifeanyi Ehirim,2009,Robert Morris,-1,-1,-1,Failure: Did not find player
4,Phillip Adams,2009,South Carolina St.,-1,-1,-1,Failure: Did not find player
5,Olawale Alesh,2009,Rhode Island,-1,-1,-1,Failure: Did not find player
6,Anthony Jones,2009,Savannah St.,Guard,198,88,Success
7,Jason Aycock,2009,UNC Wilmington,-1,-1,-1,Failure: Did not find player
8,Alex English,2009,Colgate,Forward,201,86,Success
9,Jeff Kari,2009,Lafayette,Guard,188,83,Success


In [54]:
player_file = "player_add_info_err_retry"+str(start_row)+"_"+str(end_row)+".csv"
print(player_file)
player_add_info_df.to_csv(player_file)

player_add_info_err_retry0_10000.csv


# URL of page to be scraped
url = 'https://www.sports-reference.com/cbb/players/travis-hill-1.html'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')


results = soup.find_all('div', id='info')
#print(results[0])
ptags = results[0].find_all('p')
print(ptags[0].contents[2].strip())
print(ptags[0].contents[3].contents[-1].strip())
poop = ptags[0].contents[3].contents[-1].strip().replace("(","").replace(")","").replace("cm","").replace("kg","")
print(poop)
height,weight = poop.split(",\xa0")
print(height,"-" ,weight)

#height_cm,weight_kg=tuple(ptags[0].contents[3].contents[-1].strip())
#print(height_cm)

print(ptags)
print(len(ptags))
#print(ptags[0])
#new_ptags = ptags[0].find_all('p')
print(new_ptags)
#print(new_ptags[0].text)
#print(new_ptags[0].text)
#print(new_ptags[0].text)
#print(ptags[2].text)
#print(ptags[3].text)
#print(ptags[4].text)
#for ptag in ptags:
#    print(ptag.text)

import re
results = soup.find_all('div', id='meta')
#print(results[0])
strong = results[0].find('p')
split_meta = strong.text.split(":")
#for word in split_meta:
#    print(word.strip())
print("position is",split_meta[1].strip().split(" ")[0], "and weight is")
meta2 = split_meta[1].strip().split("\n")
print(meta2[0].strip())
print(meta2[4].strip())
meta3=meta2[4].strip().split(",")
print(meta3)



for child in results[0].children:
    print("-- children--")
    print(child)
print("-- strong--")
print(strong)
print(strong.text)
spans = results[0].find_all('<span>')
print(spans)

# Retrieve the parent divs for all articles
results = soup.find_all('div', class_='players')

# loop over results to get article data
for result in results:
    # scrape the article header 
    header = result.find('h1', class_='article-item__headline').text
    
    # scrape the article subheader
    subheader = result.find('h2', class_='article-item__subheader').text
    
    # scrape the datetime
    datetime = result.find('span', class_='article-item__date')['data-date'] 
    
    # get only the date from the datetime
    date = datetime.split('T')[0]
    
    # print article data
    print('-----------------')
    print(header)
    print(subheader)
    print(date)